In [ ]:
import sys
import json
import logging
import numpy as np
import matplotlib.pyplot as plt

from qiskit import Aer, QuantumCircuit, QuantumRegister, transpile
from qiskit.ignis.mitigation.measurement import complete_meas_cal, MeasurementFilter
from qiskit.providers.ibmq.runtime.utils import RuntimeEncoder, RuntimeDecoder
#from qiskit.providers.ibmq.runtime import UserMessenger

sys.path.append('..')
from main import make_step_circuits, run_forward_circuits, run_fisc, main
from pnp_ansatze import make_pnp_ansatz
from observables import plot_counts_with_curve
from trotter import trotter_step_circuits
from sequential_minimizer import IdealCost
from hamiltonian import schwinger_model, diagonalized_evolution

## Runtime job input

In [ ]:
kwargs = {
    'num_sites': 2,
    'aJ': 1.,
    'am': 0.5,
    'omegadt': 0.2,
    'num_tsteps': 8,
    'tsteps_per_rqd': 2,
    'error_matrix': np.eye(4, dtype='f8'),
    'physical_qubits': None,
    'minimizer_shots': 4096,
    'forward_shots': 2 * 8192,
    'max_sweeps': 100
}

logging.getLogger('schwinger_rqd').setLevel(logging.INFO)

## Testing just the forward steps

In [ ]:
backend = Aer.get_backend('statevector_simulator')

num_sites = kwargs['num_sites']
aJ = kwargs['aJ']
am = kwargs['am']
omegadt = kwargs['omegadt']
num_tsteps = kwargs['num_tsteps']
physical_qubits = kwargs['physical_qubits']
error_matrix = kwargs['error_matrix']
forward_shots = kwargs['forward_shots']

_, state_labels = complete_meas_cal(qubit_list=list(range(num_sites)), qr=QuantumRegister(num_sites), circlabel='mcal')
error_mitigation_filter = MeasurementFilter(error_matrix, state_labels)

forward_step_circuits = make_step_circuits(num_sites, aJ, am, omegadt, backend, physical_qubits)

target_circuits = trotter_step_circuits(num_tsteps, forward_step_circuits, initial_state=None, measure=False)

counts_list = run_forward_circuits(target_circuits, backend, initial_layout=physical_qubits, shots=forward_shots, error_mitigation_filter=error_mitigation_filter)

plot_counts_with_curve(counts_list, num_sites, aJ, am, omegadt, num_tsteps, initial_state=None, num_toys=0)

## Testing one RQD step

In [ ]:
backend = Aer.get_backend('qasm_simulator')

num_sites = kwargs['num_sites']
aJ = kwargs['aJ']
am = kwargs['am']
omegadt = kwargs['omegadt']
tsteps_per_rqd = kwargs['tsteps_per_rqd']
max_sweeps = kwargs['max_sweeps']
minimizer_shots = kwargs['minimizer_shots']
forward_shots = kwargs['forward_shots']
physical_qubits = kwargs['physical_qubits']
error_matrix = kwargs['error_matrix']

_, state_labels = complete_meas_cal(qubit_list=list(range(num_sites)), qr=QuantumRegister(num_sites), circlabel='mcal')
error_mitigation_filter = MeasurementFilter(error_matrix, state_labels)

forward_step_circuits = make_step_circuits(num_sites, aJ, am, omegadt, backend, physical_qubits)
if not backend.configuration().simulator:
    sim_forward_step_circuits = make_step_circuits(num_sites, aJ, am, omegadt, Aer.get_backend('statevector_simulator'), physical_qubits)

if num_sites == 2:
    approximator = make_pnp_ansatz(
        num_qubits=num_sites,
        num_layers=num_sites // 2,
        initial_x_positions=[0])
elif num_sites == 4:
    approximator = make_pnp_ansatz(
        num_qubits=num_sites,
        num_layers=num_sites // 2,
        initial_x_positions=[1, 2],
        structure=[(1, 2), (0, 1), (2, 3)],
        first_layer_structure=[(0, 1), (2, 3)])

target_circuits = trotter_step_circuits(tsteps_per_rqd, forward_step_circuits, measure=False)

forward_counts = run_forward_circuits(target_circuits, backend, initial_layout=physical_qubits, shots=forward_shots, error_mitigation_filter=error_mitigation_filter)

compiler_circuit = target_circuits[-1].compose(approximator.inverse(), inplace=False)

if backend.configuration().simulator:
    ideal_cost = IdealCost(compiler_circuit)
else:
    targets = trotter_step_circuits(tsteps_per_rqd, sim_forward_step_circuits, measure=False)
    ideal_cost = IdealCost(targets[-1].compose(approximator.inverse(), inplace=False))
    
def callback_publish_sweep_result(minimizer, arg):
    sweep_result = {
        'rqd_step': 0,
        'isweep': arg['isweep'],
        'param_val': arg['sweep_param_val'],
        'cost': arg['sweep_cost'], 
        'total_shots': arg['current_shots'] + arg['sweep_shots']
    }
    print(sweep_result)

optimal_params = run_fisc(compiler_circuit, backend, physical_qubits, max_sweeps, minimizer_shots, error_mitigation_filter, ideal_cost, callback_publish_sweep_result)

In [ ]:
plt.plot(ideal_cost.shots_sweep, ideal_cost.costs_sweep)

In [ ]:
plot_counts_with_curve(forward_counts, num_sites, aJ, am, omegadt, tsteps_per_rqd, initial_state=None, num_toys=0)

vacuum_state = np.zeros(2 ** num_sites, dtype=np.complex128)
vacuum_state_index = 0
for j in range(0, num_sites, 2):
    vacuum_state_index += (1 << j)
vacuum_state[vacuum_state_index] = 1.

hamiltonian = schwinger_model(num_sites, aJ, am)
_, statevectors = diagonalized_evolution(hamiltonian, vacuum_state, omegadt * tsteps_per_rqd)
plot_counts_with_curve(forward_counts_2, num_sites, aJ, am, omegadt, tsteps_per_rqd, initial_state=statevectors[:, -1], num_toys=0)

plot_counts_with_curve(forward_counts + forward_counts_2, num_sites, aJ, am, omegadt, tsteps_per_rqd * 2, initial_state=None, num_toys=0)

## Testing the main program

In [ ]:
class UserMessenger:
    """Base class for handling communication with program users.

    This class can be used when writing a new Qiskit Runtime program.
    """
    
    def __init__(self):
        self.interim_results = []

    def publish(
            self,
            message,
            encoder=RuntimeEncoder,
            final=False
    ):
        """Publish message.

        You can use this method to publish messages, such as interim and final results,
        to the program user. The messages will be made immediately available to the user,
        but they may choose not to receive the messages.

        The `final` parameter is used to indicate whether the message is
        the final result of the program. Final results may be processed differently
        from interim results.

        Args:
            message: Message to be published. Can be any type.
            encoder: An optional JSON encoder for serializing
            final: Whether the message being published is the final result.
        """
        # pylint: disable=unused-argument
        # Default implementation for testing.
        self.interim_results.append(message)

In [ ]:
user_messenger = UserMessenger()
serialized_inputs = json.dumps(kwargs, cls=RuntimeEncoder)
deserialized_inputs = json.loads(serialized_inputs, cls=RuntimeDecoder)

forward_counts, optimal_params_list = main(backend, user_messenger, **deserialized_inputs)

In [ ]:
plot_counts_with_curve(forward_counts, num_sites, aJ, am, omegadt, num_tsteps, initial_state=None, num_toys=0)

## Testing reentry

In [ ]:
resume_rqd_step = -1
resume_from = {}

for message in user_messenger.interim_results:
    if 'optimal_params' in message:
        resume_from['optimal_params'] = message['optimal_params']
        resume_from['rqd_step'] = message['rqd_step'] + 1
        resume_rqd_step = message['rqd_step'] + 1
        
    if message['rqd_step'] == resume_rqd_step:
        if 'forward_counts' in message:
            resume_from['forward_counts'] = message['forward_counts']
        
        if 'isweep' in message and message['isweep'] == 2:
            resume_from['current_sweep'] = message['isweep'] + 1
            resume_from['param_val'] = message['param_val']
            resume_from['current_cost'] = message['cost']
            resume_from['total_shots'] = message['total_shots']
            break
            
kwargs['resume_from'] = resume_from
            
user_messenger = UserMessenger()
serialized_inputs = json.dumps(kwargs, cls=RuntimeEncoder)
deserialized_inputs = json.loads(serialized_inputs, cls=RuntimeDecoder)

forward_counts_res, optimal_params_list_res = main(backend, user_messenger, **deserialized_inputs)

In [ ]:
plot_counts_with_curve(forward_counts + forward_counts_res, num_sites, aJ, am, omegadt, num_tsteps, initial_state=None, num_toys=0)